In [1]:
import os
import sys
import json
import numpy as np
from tqdm import tqdm
sys.path.append("..")

from torch import nn
from torch.optim import Adam
from src.movientities.models.char_lstm import CharBilstm
from src.movientities.utils.processors import NerPreProcessor
from src.movientities.char_bilstm_trainer import CharBilstmTrainer
from src.movientities.data.build_dataset import CharCorpus, BuildData

In [2]:
dataset = CharCorpus(
    input_folder=f"/home/kemalaraz/Desktop/MovieEntityRecognizer/data/modified/trivia_tab_format",
    min_word_freq=3,
    batch_size=64,
)
print(f"Train set: {len(dataset.train_dataset)} sentences")
print(f"Test set: {len(dataset.test_dataset)} sentences")

Train set: 7816 sentences
Test set: 1953 sentences


In [3]:
bilstm = CharBilstm(
    input_dim=len(dataset.word_field.vocab),
    embedding_dim=300,
    char_emb_dim=25,
    char_input_dim=len(dataset.char_field.vocab),
    char_cnn_filter_num=5,
    char_cnn_kernel_size=3,
    hidden_dim=64,
    output_dim=len(dataset.tag_field.vocab),
    lstm_layers=2,
    emb_dropout=0.5,
    cnn_dropout=0.25,
    lstm_dropout=0.1,
    fc_dropout=0.25,
    word_pad_idx=dataset.word_pad_idx,
    char_pad_idx=dataset.char_pad_idx
)
bilstm.init_embeddings(
    char_pad_idx=dataset.char_pad_idx,
    word_pad_idx=dataset.word_pad_idx,
    pretrained=None,
    freeze=True
)
print(f"The model has {bilstm.count_parameters():,} trainable parameters.")
print(bilstm)

The model has 1,714,249 trainable parameters.
CharBilstm(
  (embedding): Embedding(4529, 300, padding_idx=1)
  (emb_dropout): Dropout(p=0.5, inplace=False)
  (char_emb): Embedding(39, 25, padding_idx=1)
  (char_cnn): Conv1d(25, 125, kernel_size=(3,), stride=(1,), groups=25)
  (cnn_dropout): Dropout(p=0.25, inplace=False)
  (lstm): LSTM(425, 64, num_layers=2, dropout=0.1, bidirectional=True)
  (fc_dropout): Dropout(p=0.25, inplace=False)
  (fc): Linear(in_features=128, out_features=26, bias=True)
)


In [4]:
ner = CharBilstmTrainer(
  model=bilstm,
  data=dataset,
  optimizer_cls=Adam,
  loss_fn_cls=nn.CrossEntropyLoss,
  log_file="char_bilstm_vanilla"
)
ner.train(20)

Epoch: 01 | Epoch Time: 0m 46s
	Trn Loss: 1.478 | Trn Acc: 61.20%
	Val Loss: 1.035 | Val Acc: 69.85% | Val Precision: 60.93% | Val Recall: 41.20% | Val F1 Macro: 40.82% | Val F1 Micro: 72.08%
Epoch: 02 | Epoch Time: 0m 49s
	Trn Loss: 0.781 | Trn Acc: 77.98%
	Val Loss: 0.657 | Val Acc: 80.98% | Val Precision: 72.17% | Val Recall: 71.71% | Val F1 Macro: 69.86% | Val F1 Micro: 81.53%
Epoch: 03 | Epoch Time: 1m 1s
	Trn Loss: 0.559 | Trn Acc: 83.70%
	Val Loss: 0.544 | Val Acc: 83.22% | Val Precision: 68.74% | Val Recall: 63.42% | Val F1 Macro: 62.80% | Val F1 Micro: 82.53%
Epoch: 04 | Epoch Time: 0m 58s
	Trn Loss: 0.472 | Trn Acc: 85.56%
	Val Loss: 0.543 | Val Acc: 82.79% | Val Precision: 66.66% | Val Recall: 63.03% | Val F1 Macro: 61.87% | Val F1 Micro: 81.68%
Epoch: 05 | Epoch Time: 0m 57s
	Trn Loss: 0.421 | Trn Acc: 86.83%
	Val Loss: 0.493 | Val Acc: 84.50% | Val Precision: 69.86% | Val Recall: 63.73% | Val F1 Macro: 64.15% | Val F1 Micro: 83.20%
Epoch: 06 | Epoch Time: 0m 56s
	Trn Loss: